In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

: 

In [ ]:
# import os
# # Set direktori kerja ke folder yang berisi file Anda
# os.chdir('/content/drive/My Drive/Colab Notebooks/Natural Language Processing/Task 2/')

In [ ]:
!pip install numpy pandas tensorflow nltk rouge-score

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, TimeDistributed, Masking
from sklearn.model_selection import train_test_split

In [ ]:
# Memuat dataset
df = pd.read_csv('liputan6.csv')
df = df[['clean_article', 'clean_summary']]

# Lihat contoh data
df.head()

In [ ]:
# Menampilkan jumlah total data dalam dataset
print("Total data dalam dataset:", len(df))

# Menampilkan jumlah data yang ada di setiap kolom 'clean_article' dan 'clean_summary'
print("Total data di kolom 'clean_article':", df['clean_article'].notnull().sum())
print("Total data di kolom 'clean_summary':", df['clean_summary'].notnull().sum())

# Menjumlahkan total data non-null dari kedua kolom
total_non_null = df['clean_article'].notnull().sum() + df['clean_summary'].notnull().sum()
print("Total dari kedua kolom (clean_article dan clean_summary):", total_non_null)

# Menampilkan informasi jumlah kolom dan jenis data
print("\nInfo Dataset:")
df.info()

In [ ]:
# Menggunakan Tokenizer untuk teks
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_article'].tolist() + df['clean_summary'].tolist())

# Konversi teks menjadi sequences
input_sequences = tokenizer.texts_to_sequences(df['clean_article'].tolist())
target_sequences = tokenizer.texts_to_sequences(df['clean_summary'].tolist())

# Padding sequences untuk memastikan panjangnya seragam
max_input_len = 500  # Batasi panjang maksimum input sequence
max_target_len = 50  # Batasi panjang maksimum target sequence

# Padding sequences untuk memastikan panjangnya seragam
input_sequences = pad_sequences(input_sequences, maxlen=max_input_len, padding='post')
target_sequences = pad_sequences(target_sequences, maxlen=max_target_len, padding='post')

# Mendapatkan ukuran vocabulary
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# Periksa panjang input dan target sequences yang sudah diproses
print("Panjang input sequences:", max_input_len)
print("Panjang target sequences:", max_target_len)

# Model dengan validasi
X_train, X_val, y_train, y_val = train_test_split(input_sequences, target_sequences, test_size=0.2, random_state=42)

# Menggunakan `max_input_len` untuk padding
y_train = pad_sequences(y_train, maxlen=max_input_len, padding='post')
y_val = pad_sequences(y_val, maxlen=max_input_len, padding='post')

# Model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=64))
model.add(SimpleRNN(64, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Melatih model dengan validasi
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


In [ ]:
# Plotting Loss dan Accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
def summarize_text(input_text):
    # Proses teks input
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_input_len, padding='post')

    # Prediksi output
    predicted_seq = model.predict(input_seq)
    predicted_seq = np.argmax(predicted_seq, axis=-1)[0]  # Ambil indeks dengan probabilitas tertinggi

    # Konversi output menjadi teks
    summary = ' '.join(tokenizer.index_word[idx] for idx in predicted_seq if idx > 0)
    return summary

# Mengambil input dari pengguna dan menampilkan ringkasan
input_text = input("Masukkan teks yang ingin diringkas: ")
ringkasan = summarize_text(input_text)
print("Ringkasan:", ringkasan)